# SQLAlchemy

## Installation

```bash
pip install sqlalchemy
# make sure you have a database system installed, e.g. sqlite3 using apt-get or brew etc
```

In [1]:
import sqlalchemy
from sqlalchemy import create_engine

print(sqlalchemy.__version__)
engine = create_engine('sqlite:///:memory:', echo=True) # will print all issued sql statements to the screen

1.3.11


# Schema

### Declarative Base

In [2]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

### Defining tables

In [3]:
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.types import Text, Boolean

class Author(Base):
    __tablename__ = 'authors' # the actual name in the DB

    id = Column(Integer, primary_key=True)
    first_name = Column(String, nullable=False)
    last_name = Column(String, nullable=False)

    def get_name(self):
        return "{} {}".format(self.first_name, self.last_name)

    def __repr__(self):
        return "<Author(id={}, name='{}'>".format(self.id, self.get_name())

In [4]:
from sqlalchemy.orm import relationship

class Book(Base):
    __tablename__ = 'books'

    id = Column(Integer, primary_key=True)
    title = Column(String(75), nullable=False)
    synopsis = Column(Text, nullable=True)

    author_id = Column(Integer, ForeignKey(Author.id), nullable=False)
    author = relationship("Author", backref="books")

    def __repr__(self):
        return "<Book(id={}, title='{}', author='{}', has_synopsis={})>".format(
            self.id, self.title, self.author.get_name(), self.synopsis is not None)

In [5]:
from sqlalchemy import UniqueConstraint

class Country(Base):
    __tablename__ = 'country'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False, unique=True)
    code = Column(String(3), nullable=False, unique=True)
    
    authors = relationship("Author",
                           secondary="country_author",
                           backref="countries")

    def __repr__(self):
        return "<Country(id={}, name={}, code={})>".format(self.id, self.name, self.code)

class CountryAuthor(Base):
    __tablename__ = 'country_author'
    
    author_id = Column(Integer, ForeignKey(Author.id), primary_key=True)
    country_id = Column(Integer, ForeignKey(Country.id), primary_key=True)
    native = Column(Boolean, nullable=False, default=True)
    
    #author = relationship("Author")
    #country = relationship("Country")
    
    __table_args__ = (
        UniqueConstraint("author_id", "native"),
    )

### Creating the Tables

In [6]:
Base.metadata.create_all(engine)

2019-11-13 11:33:39,549 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-11-13 11:33:39,552 INFO sqlalchemy.engine.base.Engine ()
2019-11-13 11:33:39,558 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-11-13 11:33:39,559 INFO sqlalchemy.engine.base.Engine ()
2019-11-13 11:33:39,564 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("books")
2019-11-13 11:33:39,566 INFO sqlalchemy.engine.base.Engine ()
2019-11-13 11:33:39,572 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("books")
2019-11-13 11:33:39,575 INFO sqlalchemy.engine.base.Engine ()
2019-11-13 11:33:39,580 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("authors")
2019-11-13 11:33:39,583 INFO sqlalchemy.engine.base.Engine ()
2019-11-13 11:33:39,586 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("authors")
2019-11-13 11:33:39,587 INFO sqlalchemy.engine.base.Engine ()
2019-11-13 11:33:39,58

# Data Manipulation

## Session

In [7]:
from sqlalchemy.orm import sessionmaker
SessionMaker = sessionmaker(bind=engine)
# SessionMaker can now give us sessions

## Inserting Elements

In [8]:
session = SessionMaker()

author = Author(first_name='J. K.', last_name='Rowling')
session.add(author)

country = Country(name='United Kingdom', code='UK')
session.add(country)

author.countries.append(country)
#country.authors.append(author)

session.new

IdentitySet([<Country(id=None, name=United Kingdom, code=UK)>, <Author(id=None, name='J. K. Rowling'>])

In [9]:
session.commit()

2019-11-13 11:33:40,140 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-13 11:33:40,142 INFO sqlalchemy.engine.base.Engine INSERT INTO country (name, code) VALUES (?, ?)
2019-11-13 11:33:40,143 INFO sqlalchemy.engine.base.Engine ('United Kingdom', 'UK')
2019-11-13 11:33:40,145 INFO sqlalchemy.engine.base.Engine INSERT INTO authors (first_name, last_name) VALUES (?, ?)
2019-11-13 11:33:40,146 INFO sqlalchemy.engine.base.Engine ('J. K.', 'Rowling')
2019-11-13 11:33:40,148 INFO sqlalchemy.engine.base.Engine INSERT INTO country_author (author_id, country_id, native) VALUES (?, ?, ?)
2019-11-13 11:33:40,148 INFO sqlalchemy.engine.base.Engine (1, 1, 1)
2019-11-13 11:33:40,150 INFO sqlalchemy.engine.base.Engine COMMIT


## Session -> Transactions

In [10]:
import traceback

session = SessionMaker()

author = Author(first_name='fail', last_name='failed')
country = Country(name='failure', code='UK')

session.add_all([author, country])
try:
    session.commit()
except:
    traceback.print_exc()
# session.rollback()

2019-11-13 11:33:40,222 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-13 11:33:40,223 INFO sqlalchemy.engine.base.Engine INSERT INTO authors (first_name, last_name) VALUES (?, ?)
2019-11-13 11:33:40,223 INFO sqlalchemy.engine.base.Engine ('fail', 'failed')
2019-11-13 11:33:40,225 INFO sqlalchemy.engine.base.Engine INSERT INTO country (name, code) VALUES (?, ?)
2019-11-13 11:33:40,225 INFO sqlalchemy.engine.base.Engine ('failure', 'UK')
2019-11-13 11:33:40,226 INFO sqlalchemy.engine.base.Engine ROLLBACK


Traceback (most recent call last):
  File "/home/bab/Documents/teaching/CS591L1/venv/lib/python3.5/site-packages/sqlalchemy/engine/base.py", line 1246, in _execute_context
    cursor, statement, parameters, context
  File "/home/bab/Documents/teaching/CS591L1/venv/lib/python3.5/site-packages/sqlalchemy/engine/default.py", line 581, in do_execute
    cursor.execute(statement, parameters)
sqlite3.IntegrityError: UNIQUE constraint failed: country.code

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-10-0c938d6cb6fe>", line 10, in <module>
    session.commit()
  File "/home/bab/Documents/teaching/CS591L1/venv/lib/python3.5/site-packages/sqlalchemy/orm/session.py", line 1027, in commit
    self.transaction.commit()
  File "/home/bab/Documents/teaching/CS591L1/venv/lib/python3.5/site-packages/sqlalchemy/orm/session.py", line 494, in commit
    self._prepare_impl()
  File "/home/bab/Documents/teaching/CS591L1/venv

# Query

In [11]:
session = SessionMaker()

all_authors = session.query(Author).all()
uk = session.query(Country).filter(Country.code=='UK').first()

2019-11-13 11:33:40,382 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-13 11:33:40,386 INFO sqlalchemy.engine.base.Engine SELECT authors.id AS authors_id, authors.first_name AS authors_first_name, authors.last_name AS authors_last_name 
FROM authors
2019-11-13 11:33:40,389 INFO sqlalchemy.engine.base.Engine ()
2019-11-13 11:33:40,396 INFO sqlalchemy.engine.base.Engine SELECT country.id AS country_id, country.name AS country_name, country.code AS country_code 
FROM country 
WHERE country.code = ?
 LIMIT ? OFFSET ?
2019-11-13 11:33:40,399 INFO sqlalchemy.engine.base.Engine ('UK', 1, 0)


In [12]:
print(all_authors)
print(uk)

[<Author(id=1, name='J. K. Rowling'>]
<Country(id=1, name=United Kingdom, code=UK)>


In [13]:
print(uk.authors)

2019-11-13 11:33:40,558 INFO sqlalchemy.engine.base.Engine SELECT authors.id AS authors_id, authors.first_name AS authors_first_name, authors.last_name AS authors_last_name 
FROM authors, country_author 
WHERE ? = country_author.country_id AND authors.id = country_author.author_id
2019-11-13 11:33:40,561 INFO sqlalchemy.engine.base.Engine (1,)
[<Author(id=1, name='J. K. Rowling'>]


In [14]:
from sqlalchemy.orm import joinedload
uk = session.query(Country).options(joinedload(Country.authors)).filter(Country.code=='UK').first()

print('\n')
print(uk, uk.authors)

2019-11-13 11:33:40,645 INFO sqlalchemy.engine.base.Engine SELECT anon_1.country_id AS anon_1_country_id, anon_1.country_name AS anon_1_country_name, anon_1.country_code AS anon_1_country_code, authors_1.id AS authors_1_id, authors_1.first_name AS authors_1_first_name, authors_1.last_name AS authors_1_last_name 
FROM (SELECT country.id AS country_id, country.name AS country_name, country.code AS country_code 
FROM country 
WHERE country.code = ?
 LIMIT ? OFFSET ?) AS anon_1 LEFT OUTER JOIN (country_author AS country_author_1 JOIN authors AS authors_1 ON authors_1.id = country_author_1.author_id) ON anon_1.country_id = country_author_1.country_id
2019-11-13 11:33:40,646 INFO sqlalchemy.engine.base.Engine ('UK', 1, 0)


<Country(id=1, name=United Kingdom, code=UK)> [<Author(id=1, name='J. K. Rowling'>]


## Transactions and Queries

In [15]:
session = SessionMaker()
session2 = SessionMaker()

country = Country(name='United States', code='US')
session.add(country)

countries1 = session.query(Country).all()
countries2 = session2.query(Country).all()

print(countries1)
print(countries2)

print(session.new)
print(session2.new)
session.commit()

2019-11-13 11:33:40,761 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-13 11:33:40,764 INFO sqlalchemy.engine.base.Engine INSERT INTO country (name, code) VALUES (?, ?)
2019-11-13 11:33:40,766 INFO sqlalchemy.engine.base.Engine ('United States', 'US')
2019-11-13 11:33:40,774 INFO sqlalchemy.engine.base.Engine SELECT country.id AS country_id, country.name AS country_name, country.code AS country_code 
FROM country
2019-11-13 11:33:40,777 INFO sqlalchemy.engine.base.Engine ()
2019-11-13 11:33:40,783 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-13 11:33:40,786 INFO sqlalchemy.engine.base.Engine SELECT country.id AS country_id, country.name AS country_name, country.code AS country_code 
FROM country
2019-11-13 11:33:40,789 INFO sqlalchemy.engine.base.Engine ()
[<Country(id=1, name=United Kingdom, code=UK)>, <Country(id=2, name=United States, code=US)>]
[<Country(id=1, name=United Kingdom, code=UK)>, <Country(id=2, name=United States, code=US)>]
IdentitySet([])
I

## Updating a Row

In [16]:
session = SessionMaker()

rowling = session.query(Author).first()
usa = session.query(Country).get(2)

rowling_usa = CountryAuthor(author_id=rowling.id, country_id=usa.id, native=False)
session.add(rowling_usa)

usa.name = 'United States of America'
session.add(usa)

print(session.new)
print(session.dirty)

2019-11-13 11:33:40,852 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-13 11:33:40,853 INFO sqlalchemy.engine.base.Engine SELECT authors.id AS authors_id, authors.first_name AS authors_first_name, authors.last_name AS authors_last_name 
FROM authors
 LIMIT ? OFFSET ?
2019-11-13 11:33:40,854 INFO sqlalchemy.engine.base.Engine (1, 0)
2019-11-13 11:33:40,856 INFO sqlalchemy.engine.base.Engine SELECT country.id AS country_id, country.name AS country_name, country.code AS country_code 
FROM country 
WHERE country.id = ?
2019-11-13 11:33:40,856 INFO sqlalchemy.engine.base.Engine (2,)
IdentitySet([<__main__.CountryAuthor object at 0x7f55cb5d5c18>])
IdentitySet([<Country(id=2, name=United States of America, code=US)>])


In [17]:
print(rowling.countries)

session.commit()

2019-11-13 11:33:40,967 INFO sqlalchemy.engine.base.Engine UPDATE country SET name=? WHERE country.id = ?
2019-11-13 11:33:40,969 INFO sqlalchemy.engine.base.Engine ('United States of America', 2)
2019-11-13 11:33:40,975 INFO sqlalchemy.engine.base.Engine INSERT INTO country_author (author_id, country_id, native) VALUES (?, ?, ?)
2019-11-13 11:33:40,978 INFO sqlalchemy.engine.base.Engine (1, 2, 0)
2019-11-13 11:33:40,983 INFO sqlalchemy.engine.base.Engine SELECT country.id AS country_id, country.name AS country_name, country.code AS country_code 
FROM country, country_author 
WHERE ? = country_author.author_id AND country.id = country_author.country_id
2019-11-13 11:33:40,986 INFO sqlalchemy.engine.base.Engine (1,)
[<Country(id=1, name=United Kingdom, code=UK)>, <Country(id=2, name=United States of America, code=US)>]
2019-11-13 11:33:40,992 INFO sqlalchemy.engine.base.Engine COMMIT
